# Foundation Model Prompt Engineering: Text2Text Generation


In [56]:
import logging
import json
import time
import boto3

import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm.contrib.concurrent import thread_map

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
#url = "https://yyosweaa3e.execute-api.us-east-1.amazonaws.com/LATEST/HF"
url = "https://94nlwnaute.execute-api.us-east-1.amazonaws.com/Prod/invoke"

In [57]:
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    #print(payload)
    return response

def parse_response_multiple_texts(query_response):
    model_predictions = query_response.json()
    #print(query_response)
    generated_text = model_predictions['message']
    return generated_text

### Advanced features

***
This model also supports many advanced parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

## N-shot Learning via In-context Learning

Let's put in some example input text. You can put any text containing the task, the model returns the output of the accomplished task.

Payload must be a json

### A. Zero-shot Learning

### Text summarisation - generate text and summarise text, Use cases: creating articles, writing product descriptions, etc

#### Example 1 - Generate text

In [58]:
instruction = 'Explain Australia’s foreign policy'

prompt = instruction

logger.info(f'Prompt: {prompt}')

Prompt: Explain Australia’s foreign policy
Prompt: Explain Australia’s foreign policy


In [59]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
#llm_response = response.json()["message"]
#print(llm_response)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Generated text: \n{generated_text}')

Generated text: 

Australia's foreign policy is focused on promoting and protecting Australia's national interests. These interests include security, trade, and economic prosperity. Australia works to build strong relationships with other countries and international organizations, and participates in various international forums and initiatives. Australia's foreign policy is guided by the principles of democracy, human rights, and the rule of law.
Generated text: 

Australia's foreign policy is focused on promoting and protecting Australia's national interests. These interests include security, trade, and economic prosperity. Australia works to build strong relationships with other countries and international organizations, and participates in various international forums and initiatives. Australia's foreign policy is guided by the principles of democracy, human rights, and the rule of law.


#### Example 2 - Text summarisation

In [12]:
context = generated_text
logger.info(f'Context: \n{context}')

Context: 

Australia's foreign policy is guided by the principles of international cooperation, respect for international law, and the promotion of peace, security, and prosperity. Australia is a middle power that works closely with its allies and partners to promote international stability and resolve conflicts. Australia is also a strong advocate for human rights, gender equality, and sustainable development.


In [15]:
instruction = "Explain the above in one sentence"

prompt = f'{context}\n{instruction}:'
logger.info(f'Prompt: {prompt}')

Prompt: 
Australia's foreign policy is guided by the principles of international cooperation, respect for international law, and the promotion of peace, security, and prosperity. Australia is a middle power that works closely with its allies and partners to promote international stability and resolve conflicts. Australia is also a strong advocate for human rights, gender equality, and sustainable development.
Explain the above in one sentence:


In [17]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Generated text: \n{generated_text}')

Gnerated text: 

Australia is a middle power that works closely with its allies and partners to promote international stability and resolve conflicts.


#### Sentiment Analysis/Classification

#### Example 3: generate text, product description

In [22]:
instruction = "Generate a concise product description for the product: laptop."

prompt = f'{instruction}:'
logger.info(f'Prompt: {prompt}')

Prompt: Generate a concise product description for the product: laptop.:


In [23]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Generated text: \n{generated_text}')

Generated text: 

A laptop is a portable computer that is designed to be easily carried and used. It can be used for a variety of tasks, including word processing, web browsing, and playing games.


#### Example 4: Lets control the output of product description, indicating the sections you would like the model to generate

In [24]:
instruction = "Generate a concise product description for the product: laptop."
output_indicator = "Use the following format: Hook, Solution, Features and Benefits, Call to Action."

prompt = f'{instruction}\n{output_indicator}'
logger.info(f'Prompt: {prompt}')

Prompt: Generate a concise product description for the product: laptop.
Use the following format: Hook, Solution, Features and Benefits, Call to Action.


In [25]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Generated text: \n{generated_text}')

Generated text: 

Hook:
A sleek, powerful laptop that is perfect for on-the-go use.
Solution:
Our laptop features a long battery life, a fast processor, and a lightweight design.
Features and Benefits:
-Long battery life: Up to 10 hours of use on a single charge.
-Fast processor: Perfect for multitasking and streaming.
-Lightweight design: Easily portable and perfect for on-the-go use.
Call to Action:
Order your laptop today!


### 2. Information extraction: Extract keywords, location, people, sentiment. Use cases: SEO, metadata extraction

The example is an excerpt from the article : https://www.sbs.com.au/news/article/anthony-albanese-to-hold-talks-with-joe-biden-and-rishi-sunak-on-aukus-defence-pact/0ar8pshdb


#### Extract location from the article

In [38]:
article_context = "US President Joe Biden will host leaders of Australia and the United Kingdom \
in San Diego next week to chart a way forward for the provision of nuclear-powered submarines and \
other high-tech weaponry to Australia, sources familiar with the plans said. \
The spokesperson for UK Prime Minister Rishi Sunak said he will visit the United States on Monday \
to meet Mr Biden and Prime Minister Anthony Albanese for talks on the AUKUS defence agreement."


In [39]:
instruction = "List the locations mentioned in the paragraph above:"

prompt = f'{article_context}\n{instruction}'
logger.info(f'Prompt: {prompt}')

Prompt: US President Joe Biden will host leaders of Australia and the United Kingdom in San Diego next week to chart a way forward for the provision of nuclear-powered submarines and other high-tech weaponry to Australia, sources familiar with the plans said. The spokesperson for UK Prime Minister Rishi Sunak said he will visit the United States on Monday to meet Mr Biden and Prime Minister Anthony Albanese for talks on the AUKUS defence agreement.
List the locations mentioned in the paragraph above:


In [40]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Locations: \n{generated_text}')

Generated text: 

San Diego, United States, Australia, United Kingdom


#### Extract people from the article

In [41]:
instruction = "List the people mentioned in the paragraph above:"

prompt = f'{article_context}\n{instruction}'
logger.info(f'Prompt: {prompt}')

Prompt: US President Joe Biden will host leaders of Australia and the United Kingdom in San Diego next week to chart a way forward for the provision of nuclear-powered submarines and other high-tech weaponry to Australia, sources familiar with the plans said. The spokesperson for UK Prime Minister Rishi Sunak said he will visit the United States on Monday to meet Mr Biden and Prime Minister Anthony Albanese for talks on the AUKUS defence agreement.
List the people mentioned in the paragraph above:


In [44]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'People mentined in the article: \n{generated_text}')

People mentined in the article: 

Biden, Sunak, Albanese


#### Extract keywords from the article

In [45]:
instruction = "Extract keywords from the paragraph above:"

prompt = f'{article_context}\n{instruction}'
logger.info(f'Prompt: {prompt}')

Prompt: US President Joe Biden will host leaders of Australia and the United Kingdom in San Diego next week to chart a way forward for the provision of nuclear-powered submarines and other high-tech weaponry to Australia, sources familiar with the plans said. The spokesperson for UK Prime Minister Rishi Sunak said he will visit the United States on Monday to meet Mr Biden and Prime Minister Anthony Albanese for talks on the AUKUS defence agreement.
Extract keywords from the paragraph above:


In [46]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Keywords in the article: \n{generated_text}')

Keywords in the article: 

Australia, UK, US, nuclear-powered submarines, AUKUS


#### Extract sentiment from the article

In [53]:
article_context = 'Prime Minister Anthony Albanese is expected to announce Australias pathway to getting nuclear-powered subs in San Diego on Tuesday (AEDT) alongside Mr Biden and Mr Sunak. He will then hold bilateral meetings with both. \
"This is a joint arrangement between Australia, the United States, and the United Kingdom," he told reporters on Thursday during a visit to India. \
"We are great friends. We have over a century of standing side by side during peacetime and during conflict.'


In [54]:
instruction = "Extract the sentiment from the paragraph above and classify into neutral, positive or negative"

prompt = f'{article_context}\n{instruction}'
logger.info(f'Prompt: {prompt}')

Prompt: Prime Minister Anthony Albanese is expected to announce Australias pathway to getting nuclear-powered subs in San Diego on Tuesday (AEDT) alongside Mr Biden and Mr Sunak. He will then hold bilateral meetings with both. "This is a joint arrangement between Australia, the United States, and the United Kingdom," he told reporters on Thursday during a visit to India. "We are great friends. We have over a century of standing side by side during peacetime and during conflict.
Extract the sentiment from the paragraph above and classify into neutral, positive or negative


In [55]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Sentiment of the article: \n{generated_text}')

Keywords in the article: 

Prime Minister Albanese is expected to announce Australia's pathway to getting nuclear-powered subs in San Diego on Tuesday, which is considered a positive change.


#### 3. Question answering from the article

In [66]:
article_context = "Novak Djokovic reaches record 23 grand slam titles after French Open final win\n\nSerb beats Casper Ruud 7-6 (1), 6-3, 7-5 at Roland Garros\nDjokovic now has one more grand slam title than Rafael Nadal\n\nSince his emergence at the top level of professional tennis 18 years ago, Novak Djokovic has used the dizzying bar set by Roger Federer and Rafael Nadal before him as inspiration to push himself to his limits, never doubting that he would one day rise above it.\n\nWhat once seemed impossible eventually became inevitable. On Sunday, Djokovic finally surpassed his great rivals in the most significant category of all as he defeated Casper Ruud 7-6 (1), 6-3, 7-5 to clinch his 23rd grand slam title, breaking his tie of 22 with Nadal.\n"

In [67]:
question = "Whom did Djokovic defeat and what was the score?"

In [68]:
prompt = f'{article_context}\n{question}'
logger.info(f'Prompt: {prompt}')

Prompt: Novak Djokovic reaches record 23 grand slam titles after French Open final win

Serb beats Casper Ruud 7-6 (1), 6-3, 7-5 at Roland Garros
Djokovic now has one more grand slam title than Rafael Nadal

Since his emergence at the top level of professional tennis 18 years ago, Novak Djokovic has used the dizzying bar set by Roger Federer and Rafael Nadal before him as inspiration to push himself to his limits, never doubting that he would one day rise above it.

What once seemed impossible eventually became inevitable. On Sunday, Djokovic finally surpassed his great rivals in the most significant category of all as he defeated Casper Ruud 7-6 (1), 6-3, 7-5 to clinch his 23rd grand slam title, breaking his tie of 22 with Nadal.

Whom did Djokovic defeat and what was the score?
Prompt: Novak Djokovic reaches record 23 grand slam titles after French Open final win

Serb beats Casper Ruud 7-6 (1), 6-3, 7-5 at Roland Garros
Djokovic now has one more grand slam title than Rafael Nadal

S

In [69]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Answer: \n{generated_text}')

Answer: 

Novak Djokovic defeated Casper Ruud 7-6 (1), 6-3, 7-5 to win the French Open and clinch his 23rd grand slam title.
Answer: 

Novak Djokovic defeated Casper Ruud 7-6 (1), 6-3, 7-5 to win the French Open and clinch his 23rd grand slam title.


#### 4. K-Shot prompt example

#### Example - Lets check the model output without providing the examples of whatwe expect the model to respond back

In [78]:
prompt = f"""
Review: "I really had fun watching this movie"
This review is:

"""
logger.info(f'Prompt: {prompt}')

Prompt: 
Review: "I really had fun watching this movie"
This review is:


Prompt: 
Review: "I really had fun watching this movie"
This review is:




In [79]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Answer: \n{generated_text}')

Answer: 
Helpful
Not helpful

Please rate this review:

Excellent
Good
Average
Poor
Answer: 
Helpful
Not helpful

Please rate this review:

Excellent
Good
Average
Poor


#### Example 2: 
As you can see above, we might expect the model to respond back with positive, nuetral or negative.
But it does not and responds back with a more generic outputlets see how we can provide some examples to the model on what we expect 

In [76]:
prompt = f"""

Review: "I loved this movie!"
This review is positive.

Review: "I am not sure, I think the movie was fine."
The review is neutral.

Review: "This movie was a waste of time and money."
This review is negative.

Review: "I really had fun watching this movie"
This review is:

"""
logger.info(f'Prompt: {prompt}')

Prompt: 

Review: "I loved this movie!"
This review is positive.

Review: "I am not sure, I think the movie was fine."
The review is neutral.

Review: "This movie was a waste of time and money."
This review is negative.

Review: "I really had fun watching this movie"
This review is:


Prompt: 

Review: "I loved this movie!"
This review is positive.

Review: "I am not sure, I think the movie was fine."
The review is neutral.

Review: "This movie was a waste of time and money."
This review is negative.

Review: "I really had fun watching this movie"
This review is:




In [77]:
payload = {"prompt":prompt, "max_token":100, "temperature": 0.5}
response = query_endpoint_with_json_payload(url, payload)
generated_text = parse_response_multiple_texts(response)
logger.info(f'Answer: \n{generated_text}')

Answer: 
positive
Answer: 
positive
